In [36]:
import pandas as pd
import numpy as np
import re
import string
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report

In [46]:
import pandas as pd

train_df = pd.read_csv("twitter_training.csv", header=None)
val_df = pd.read_csv("twitter_validation.csv", header=None)

print(train_df.head())  # Check the first few rows


# Check dataset structure

      0            1         2  \
0  2401  Borderlands  Positive   
1  2401  Borderlands  Positive   
2  2401  Borderlands  Positive   
3  2401  Borderlands  Positive   
4  2401  Borderlands  Positive   

                                                   3  
0  im getting on borderlands and i will murder yo...  
1  I am coming to the borders and I will kill you...  
2  im getting on borderlands and i will kill you ...  
3  im coming on borderlands and i will murder you...  
4  im getting on borderlands 2 and i will murder ...  


In [48]:
# Load dataset with correct column names
train_df = pd.read_csv("twitter_training.csv", header=None, names=["ID", "Product", "Sentiment", "Review"])
val_df = pd.read_csv("twitter_validation.csv", header=None, names=["ID", "Product", "Sentiment", "Review"])

# Drop unnecessary columns (ID and Product)
train_df = train_df[["Sentiment", "Review"]]
val_df = val_df[["Sentiment", "Review"]]

print(train_df.head())  # Verify if the columns are correct now


  Sentiment                                             Review
0  Positive  im getting on borderlands and i will murder yo...
1  Positive  I am coming to the borders and I will kill you...
2  Positive  im getting on borderlands and i will kill you ...
3  Positive  im coming on borderlands and i will murder you...
4  Positive  im getting on borderlands 2 and i will murder ...


In [49]:
train_df.dropna(inplace=True)
val_df.dropna(inplace=True)


In [50]:
# Count occurrences of each sentiment in training data
sentiment_counts_train = train_df["Sentiment"].value_counts()

# Count occurrences of each sentiment in validation data
sentiment_counts_val = val_df["Sentiment"].value_counts()

# Print the counts
print("Training Data Sentiment Distribution:\n", sentiment_counts_train)
print("\nValidation Data Sentiment Distribution:\n", sentiment_counts_val)


Training Data Sentiment Distribution:
 Sentiment
Negative      22358
Positive      20655
Neutral       18108
Irrelevant    12875
Name: count, dtype: int64

Validation Data Sentiment Distribution:
 Sentiment
Neutral       285
Positive      277
Negative      266
Irrelevant    172
Name: count, dtype: int64


In [51]:
import re
import pandas as pd

# Function to preprocess text
def preprocess_text(text):
    if isinstance(text, str):  # Ensure text is a string
        text = text.lower()  # Convert to lowercase
        text = re.sub(r"http\S+|www\S+|https\S+", "", text)  # Remove URLs
        text = re.sub(r"@\w+|\#", "", text)  # Remove mentions/hashtags
        text = re.sub(r"[^a-z\s]", "", text)  # Remove special characters and numbers
        text = re.sub(r"\s+", " ", text).strip()  # Remove extra spaces
    else:
        text = ""  # Handle NaN cases
    return text

# Apply preprocessing
train_df["Review"] = train_df["Review"].apply(preprocess_text)
val_df["Review"] = val_df["Review"].apply(preprocess_text)

# Check data after cleaning
print(train_df.head())
print(val_df.head())


  Sentiment                                             Review
0  Positive  im getting on borderlands and i will murder yo...
1  Positive  i am coming to the borders and i will kill you...
2  Positive  im getting on borderlands and i will kill you all
3  Positive  im coming on borderlands and i will murder you...
4  Positive  im getting on borderlands and i will murder yo...
    Sentiment                                             Review
0  Irrelevant  i mentioned on facebook that i was struggling ...
1     Neutral  bbc news amazon boss jeff bezos rejects claims...
2    Negative  why do i pay for word when it functions so poo...
3    Negative  csgo matchmaking is so full of closet hacking ...
4     Neutral  now the president is slapping americans in the...


In [52]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Initialize the TF-IDF Vectorizer
vectorizer = TfidfVectorizer(max_features=5000)  # Limits to 5000 most important words

# Fit and transform the training data, only transform the validation data
X_train = vectorizer.fit_transform(train_df["Review"])
X_val = vectorizer.transform(val_df["Review"])

# Convert labels into numerical format
y_train = train_df["Sentiment"]
y_val = val_df["Sentiment"]

# Check the shape of the vectorized data
print("Shape of X_train:", X_train.shape)
print("Shape of X_val:", X_val.shape)


Shape of X_train: (73996, 5000)
Shape of X_val: (1000, 5000)


In [57]:
import joblib

# Save the trained model
joblib.dump(svm_model, "sentiment_shrey1.pkl")

# Save the vectorizer
joblib.dump(vectorizer, "vectorizer1.pkl")

print("Model and vectorizer saved successfully!")


Model and vectorizer saved successfully!


Loading and Testing


In [ ]:

import joblib

# Load the trained model and vectorizer
svm_model = joblib.load("sentiment_shrey1.pkl")
vectorizer = joblib.load("vectorizer1.pkl")

print("Model and vectorizer loaded successfully!")

def predict_sentiment(text):
    processed_text = vectorizer.transform([text])  # Convert text to features
    prediction = svm_model.predict(processed_text)  # Get sentiment prediction
    return prediction[0]  # Return predicted sentiment

# Sample test cases
sample_reviews = [
    "I love this product! It's amazing.",  # Expected: Positive
    "This is the worst experience I've ever had.",  # Expected: Negative
    "It's okay, not great but not bad either.",  # Expected: Neutral
    "I don't think this is relevant to the topic.",  # Expected: Irrelevant
]

# Predict sentiment for each review
for review in sample_reviews:
    print(f"Review: {review}\nPredicted Sentiment: {predict_sentiment(review)}\n")


Model and vectorizer loaded successfully!
Review: I love this product! It's amazing.
Predicted Sentiment: Positive

Review: This is the worst experience I've ever had.
Predicted Sentiment: Negative

Review: It's okay, not great but not bad either.
Predicted Sentiment: Negative

Review: I don't think this is relevant to the topic.
Predicted Sentiment: Irrelevant



In [4]:
import joblib
import pandas as pd

# Load the trained model and vectorizer
svm_model = joblib.load("sentiment_shrey1.pkl")
vectorizer = joblib.load("vectorizer1.pkl")


In [5]:
# Function to load dataset
def load_dataset(name):
    if name == "Twitter Dataset":
        df = pd.read_csv("twitter_validation.csv", names=["ID", "Product", "Sentiment", "Review"])
        df = df[["Sentiment", "Review"]]
    else:
        df = pd.read_csv("amazon_reviews.csv")
        df = df.rename(columns={"Score": "Sentiment", "Text": "Review"})
        df = df[["Sentiment", "Review"]]
        sentiment_map = {1: "Negative", 2: "Negative", 3: "Neutral", 4: "Positive", 5: "Positive"}
        df["Sentiment"] = df["Sentiment"].map(sentiment_map)
    return df

# Load the validation dataset
test_df = load_dataset("Twitter Dataset")
X_val = test_df["Review"]
y_val = test_df["Sentiment"]


In [10]:
# Define your sentiment mapping
sentiment_map = {
    1: "Negative", 
    2: "Negative", 
    3: "Neutral", 
    4: "Positive", 
    5: "Positive"
}

# Load dataset
def load_dataset(name):
    if name == "Twitter Dataset":
        df = pd.read_csv("twitter_validation.csv", names=["ID", "Product", "Sentiment", "Review"])
        df = df[["Sentiment", "Review"]]
    else:
        df = pd.read_csv("amazon_reviews.csv")
        df = df.rename(columns={"Score": "Sentiment", "Text": "Review"})
        df = df[["Sentiment", "Review"]]
        sentiment_map = {1: "Negative", 2: "Negative", 3: "Neutral", 4: "Positive", 5: "Positive"}
        # Map Sentiment and filter out unknown labels
        df["Sentiment"] = df["Sentiment"].map(sentiment_map)
        
        # Filter out rows where sentiment is NaN (invalid labels)
        df = df.dropna(subset=["Sentiment"])

    return df

# Function to predict sentiment
def predict_sentiment(review):
    input_vector = vectorizer.transform([review])
    return svm_model.predict(input_vector)[0]
